In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv('polarity_labeling.csv')
data.head()

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,asin,date,product,review,score,user,CS_SS,target,feature,polarity
0,B00000JFOA,"01 6, 2000",c-2000,the newer model c-2020 has all the fixes exce...,4.0,Lee,CS,c-2020,lens,0
1,B000023VQC,"06 24, 2001",pdc-700,it likes to change from xga to vga etc.. *the ...,2.0,A. Gray,CS,vga,battery,0
2,B00003OPEY,"04 26, 2000",dsc-f505,the included memory stick is a little small b...,5.0,David Pirmann,CS,110,memory,1
3,B00004R8V6,"09 5, 2000",d-360l,it also adds a nifty little optical function ...,5.0,"Baltic Books ""Vic""",CS,d-460,"optical,battery,lcd,compression,flash",1
4,B00004R8VC,"04 18, 2000",d-460,i read a lot of reviews on the d450 optical be...,5.0,JR,CS,d450,optical,-1


In [4]:
def Make_matrix(df):
    # unique()는 특정 시리즈 데이터에서 유일한 값들을 넘파이배열로 출력
    product_list = df['product'].unique()
    matrix = np.zeros((len(product_list), len(product_list)), dtype=np.float32)
    
    # 불필요한 데이터 제거
    df = df[df['polarity'] != 0]
    for i, p in enumerate(product_list):
        df_re = df[df['product'] == p]
        df_cs = df_re[df_re['CS_SS'] == 'CS']
        df_ss = df_re[df_re['CS_SS'] == 'SS']
        
        if i%100 == 0:
            print(i)
        
        for j, t in enumerate(product_list):
            if i == j:
                # ss에 의한 웨이트 계산
                ss_data = df_ss['polarity'].value_counts()
                if len(ss_data.index) == 2:
                    matrix[i,j] = int(ss_data[ss_data.index == 1]) / int(ss_data[ss_data.index == -1])
                elif 1 not in ss_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = int(ss_data[ss_data.index == 1])

            else:
                # cs에 의한 웨이트 계산
                cs_data = df_cs[df_cs['target'].str.contains(t) == True]['polarity'].value_counts()
                if len(cs_data.index) == 2:
                    matrix[i,j] = int(cs_data[cs_data.index == 1]) / int(cs_data[cs_data.index == -1])
                elif 1 not in cs_data.index:
                    matrix[i,j] = 0
                else:
                    matrix[i,j] = int(cs_data[cs_data.index == 1])
    
    # 매트릭스를 데이터프레임형식으로 바꿈
    dff = pd.DataFrame(matrix, index = product_list, columns = product_list)
    dff.loc['sum'] = dff.sum()

    return dff

In [8]:
def Make_Rank(df, d = 0.85, MAX_ITERATIONS = 50, threshold = 0.0001):
    # sum = 0 인 열 전체를 지우는 함수
    df = df.T
    df=df[df['sum'] != 0].T
    
    product_list = df.columns
    
    # 초기 스코어 설정( 1 )
    score = np.ones(len(product_list))
    
    # 최대 반복 횟수 50 설정
    for iter in range(0,MAX_ITERATIONS):
        prev_score = np.copy(score)
        
        for i, p in enumerate(product_list):
            # 넘파이 배열의 사칙연산을 사용하여 for문을 줄임
            score[i] = (1-d) + d*sum(df.loc[p] / df.loc['sum'] * prev_score)

        # 반복 중 수치 변화가 일정 값 미만으로 떨어지면 코드 종료
        if np.sum(np.fabs(prev_score-score)) <= threshold:
            print('요기서 끝 :', iter)
            break
            
    normal_score = score / np.sqrt((np.sum(score**2)))
    dff = pd.DataFrame(normal_score, index = product_list)
    return dff

In [57]:
def Make_Feature_Rank_Labeling(df):
    f_list = ['pixel','lens','optical','memory','burst']
    product_list = df['product'].unique()
    result = pd.DataFrame(index = product_list)
    
    for f in f_list:
        dff = df[df.feature.str.contains(f)]
        matrix = Make_matrix(dff)
        matrix.to_excel('{}_matrix.xlsx'.format(f), index = False)
        score = Make_Rank(matrix)
        score.columns = [f]
        
        result.join(score)
        
    result = result.fillna(0)
    return result

In [ ]:
result = Make_Feature_Rank_Labeling(data)
result.head()

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
요기서 끝 : 45
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [14]:
data2 = pd.read_excel('pixel_matrix.xlsx')
data2.head()

,Unnamed: 0,d-460,qv3000ex,s10,pdr-m60,c-3000,s100,mvc-fd95,40i,d-490,...,p7800,aw1,ek-gn120zkaxar,l27,sx600,l830,wb35f,elph-150,g1x,hx400v/b
0,d-460,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,qv3000ex,0,6,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,s10,0,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,pdr-m60,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,c-3000,0,0,0,0,2,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
data2.tail()

,d-460,qv3000ex,s10,pdr-m60,c-3000,s100,mvc-fd95,40i,d-490,d30,...,p7800,aw1,ek-gn120zkaxar,l27,sx600,l830,wb35f,elph-150,g1x,hx400v/b
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
wb35f,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
elph-150,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
g1x,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
hx400v/b,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
sum,9,6,20,1,2,27,4,1,7,23,...,0,2,0,0,1,1,0,1,7,0


In [24]:
aa = Make_Rank(data2)
aa.head()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
요기서 끝 : 45


,0
d-460,0.013170
qv3000ex,0.050806
s10,0.006440
pdr-m60,0.029754
c-3000,0.053824


In [36]:
aa.columns = ['a']
aa.head()

,a
d-460,0.013170
qv3000ex,0.050806
s10,0.006440
pdr-m60,0.029754
c-3000,0.053824


In [30]:
len(aa)

858

In [54]:
p_list = data2.columns
a = pd.DataFrame(index = p_list)
b = a.join(aa)
b.tail()

,a
l830,0.029754
wb35f,NaN
elph-150,0.029754
g1x,0.005080
hx400v/b,NaN


In [56]:
c = b.fillna(0)
c

,a
d-460,0.013170
qv3000ex,0.050806
s10,0.006440
pdr-m60,0.029754
c-3000,0.053824
s100,0.009846
mvc-fd95,0.029754
40i,0.029754
d-490,0.016872
d30,0.006012


In [43]:
a.head()

,a,b,c
d-460,0,0,0
qv3000ex,0,0,0
s10,0,0,0
pdr-m60,0,0,0
c-3000,0,0,0


In [ ]:
help(merge)

In [46]:
a.apply(lambda x: x['a'] = aa['a'])

SyntaxError: lambda cannot contain assignment (<ipython-input-46-8de0a6e203f8>, line 1)

In [39]:
help(a.join)

Help on method join in module pandas.core.frame:

join(other, on=None, how='left', lsuffix='', rsuffix='', sort=False) method of pandas.core.frame.DataFrame instance
    Join columns of another DataFrame.
    
    Join columns with `other` DataFrame either on index or on a key
    column. Efficiently join multiple DataFrame objects by index at once by
    passing a list.
    
    Parameters
    ----------
    other : DataFrame, Series, or list of DataFrame
        Index should be similar to one of the columns in this one. If a
        Series is passed, its name attribute must be set, and that will be
        used as the column name in the resulting joined DataFrame.
    on : str, list of str, or array-like, optional
        Column or index level name(s) in the caller to join on the index
        in `other`, otherwise joins index-on-index. If multiple
        values given, the `other` DataFrame must have a MultiIndex. Can
        pass an array as the join key if it is not already contai